In [1]:
import matchzoo as mz
from matchzoo import DataGenerator
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import scipy

from sklearn.metrics import precision_recall_fscore_support

Using TensorFlow backend.


# Load Data and Preprocess It

In [2]:
train_raw = mz.pack(pd.read_csv('matchzoo experiments/toy_matchzoo_train.csv', index_col=0))
test_raw = mz.pack(pd.read_csv('matchzoo experiments/toy_matchzoo_test.csv', index_col=0))

train_raw.relation['label'] = train_raw.relation['label'].astype('float32')
test_raw.relation['label'] = test_raw.relation['label'].astype('float32')

preprocessor = mz.preprocessors.DSSMPreprocessor()

preprocessor.fit(train_raw)
preprocessor.context

train_preprocessed = preprocessor.transform(train_raw)
test_preprocessed = preprocessor.transform(test_raw)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|█| 716/716 [00:00<00:00, 9070.57it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|█| 1025/1025 [00:00<00:00, 3849.38it/s]
Building VocabularyUnit from a datapack.: 100%|███████████████████████████| 112803/112803 [00:00<00:00, 3133436.26it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|█| 716/716 [00:00<00:00, 6404.19it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|█| 1025/1025 [00:00<00:00, 2840.78it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit 

# Build Model

In [3]:
model = mz.models.DSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()

Parameter "name" set to DSSM.
Parameter "task" set to Ranking Task.


Since no loss function set, default is Mean Square Error

In [4]:
model.build()
model.compile()
print(model._params['task'].loss)


mse


# Train Model


In [5]:
x_train, y_train = train_preprocessed.unpack()
x_test, y_test = test_preprocessed.unpack()

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, verbose=2)

Train on 4986 samples, validate on 1248 samples
Epoch 1/20
 - 4s - loss: 0.1791 - val_loss: 0.1581
Epoch 2/20
 - 4s - loss: 0.1545 - val_loss: 0.1510
Epoch 3/20
 - 4s - loss: 0.1484 - val_loss: 0.1461
Epoch 4/20
 - 4s - loss: 0.1444 - val_loss: 0.1428
Epoch 5/20
 - 4s - loss: 0.1418 - val_loss: 0.1409
Epoch 6/20
 - 4s - loss: 0.1403 - val_loss: 0.1399
Epoch 7/20
 - 4s - loss: 0.1396 - val_loss: 0.1393
Epoch 8/20
 - 4s - loss: 0.1392 - val_loss: 0.1391
Epoch 9/20
 - 4s - loss: 0.1390 - val_loss: 0.1390
Epoch 10/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 11/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 12/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 13/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 14/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 15/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 16/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 17/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 18/20
 - 4s - loss: 0.1389 - val_loss: 0.1389
Epoch 19/20
 - 4s - loss:

# Evaluate

In [6]:
y_pred = model.predict(x_test)

print(y_pred[0:10])
print(y_test[0:10])





[[0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]
 [0.16652368]]
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]]


In [7]:
#classify
y_pred_cutoff = y_pred > .5

prec, recall, f1beta, support = precision_recall_fscore_support(y_test, y_pred_cutoff, pos_label=1, average='binary')

print(prec)
print(recall)
print(f1beta)
print(f1_score(y_test, y_pred_cutoff, pos_label=1, average='binary'))

C:\Users\patelr9\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0
0.0
0.0


C:\Users\patelr9\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0
